In [2]:
import torch.nn as nn

In [3]:
class Generator(nn.Module):
    def __init__(self, opt):
        super().__init__()

        self.init_size = opt.img_size // 4
        self.l1 = nn.Sequential(nn.Linear(opt.latent_dim,
                                128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, opt.channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


In [4]:
class Discriminator(nn.Module):
    def __init__(self, opt):
        super().__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1),
                     nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(opt.channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = opt.img_size // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1))

    def forward(self, img):
        features = self.forward_features(img)
        validity = self.adv_layer(features)
        return validity

    def forward_features(self, img):
        features = self.model(img)
        features = features.view(features.shape[0], -1)
        return features


In [5]:
class Encoder(nn.Module):
    def __init__(self, opt):
        super().__init__()

        def encoder_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1),
                     nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *encoder_block(opt.channels, 16, bn=False),
            *encoder_block(16, 32),
            *encoder_block(32, 64),
            *encoder_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = opt.img_size // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2,
                                                 opt.latent_dim),
                                       nn.Tanh())

    def forward(self, img):
        features = self.model(img)
        features = features.view(features.shape[0], -1)
        validity = self.adv_layer(features)
        return validity


In [14]:
import torch
from torch.utils.data import DataLoader, Dataset
from PIL import Image, ImageDraw, ImageOps
import os
import numpy as np
import torchvision.transforms as transforms
import pandas as pd
import torch.nn as nn
from torchvision.utils import save_image
import torch.autograd as autograd
import re

In [15]:
# Reference; https://deep-learning-study.tistory.com/470
class BreastDataset(Dataset):
    def __init__(self, data_dir, transform, data_type='train'):
        # path to images
        path2data = os.path.join(data_dir, data_type)

        # get a list of images
        filenames = os.listdir(path2data)

        # get the full path to images
        self.full_filenames = [os.path.join(path2data, f) for f in filenames]
        self.transform = transform

    def __len__(self):
        # return size of dataset
        return len(self.full_filenames)

    def __getitem__(self, idx):
        # open image, apply transforms and return with label
        image = ImageOps.grayscale(Image.open(self.full_filenames[idx]))
        image=image.convert('RGB')
        #image = Image.open(self.full_filenames[idx]).convert('RGB')
        image = self.transform(image)
        return image, int(re.findall("\d+",self.full_filenames[idx])[0])
        # return image, self.labels[idx]

In [16]:
# collect all code/function, deleted label varaible within the code

In [17]:
def compute_gradient_penalty(D, real_samples, fake_samples, device):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = torch.rand(*real_samples.shape[:2], 1, 1, device=device)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + (1 - alpha) * fake_samples)
    interpolates = autograd.Variable(interpolates, requires_grad=True)
    d_interpolates = D(interpolates)
    fake = torch.ones(*d_interpolates.shape, device=device)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(outputs=d_interpolates, inputs=interpolates,
                              grad_outputs=fake, create_graph=True,
                              retain_graph=True, only_inputs=True)[0]
    gradients = gradients.view(gradients.shape[0], -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [18]:
def train_wgangp(opt, generator, discriminator,dataloader, device, lambda_gp=10):
    generator.to(device)
    discriminator.to(device)

    optimizer_G = torch.optim.Adam(generator.parameters(),
                                   lr=opt.lr, betas=(opt.b1, opt.b2))
    optimizer_D = torch.optim.Adam(discriminator.parameters(),
                                   lr=opt.lr, betas=(opt.b1, opt.b2))

    os.makedirs("results/images", exist_ok=True)

    padding_epoch = len(str(opt.n_epochs))
    padding_i = len(str(len(dataloader)))

    batches_done = 0
    for epoch in range(opt.n_epochs):
        for i, (imgs, _)in enumerate(dataloader):

            # Configure input
            real_imgs = imgs.to(device)

            # ---------------------
            #  Train Discriminator
            # ---------------------

            optimizer_D.zero_grad()

            # Sample noise as generator input
            z = torch.randn(imgs.shape[0], opt.latent_dim, device=device)

            # Generate a batch of images
            fake_imgs = generator(z)

            # Real images
            real_validity = discriminator(real_imgs)
            # Fake images
            fake_validity = discriminator(fake_imgs.detach())
            # Gradient penalty
            gradient_penalty = compute_gradient_penalty(discriminator,
                                                        real_imgs.data,
                                                        fake_imgs.data,
                                                        device)
            # Adversarial loss
            d_loss = (-torch.mean(real_validity) + torch.mean(fake_validity)
                      + lambda_gp * gradient_penalty)

            d_loss.backward()
            optimizer_D.step()

            optimizer_G.zero_grad()

            # Train the generator and output log every n_critic steps
            if i % opt.n_critic == 0:

                # -----------------
                #  Train Generator
                # -----------------

                # Generate a batch of images
                fake_imgs = generator(z)
                # Loss measures generator's ability to fool the discriminator
                # Train on fake images
                fake_validity = discriminator(fake_imgs)
                g_loss = -torch.mean(fake_validity)

                g_loss.backward()
                optimizer_G.step()

                print(f"[Epoch {epoch:{padding_epoch}}/{opt.n_epochs}] "
                      f"[Batch {i:{padding_i}}/{len(dataloader)}] "
                      f"[D loss: {d_loss.item():3f}] "
                      f"[G loss: {g_loss.item():3f}]")

                if batches_done % opt.sample_interval == 0:
                    save_image(fake_imgs.data[:25],
                               f"results/images/{batches_done:06}.png",
                               nrow=5, normalize=True)

                batches_done += opt.n_critic

    torch.save(generator.state_dict(), "results/generator")
    torch.save(discriminator.state_dict(), "results/discriminator")

In [19]:
def train_encoder_izif(opt, generator, discriminator, encoder, dataloader, device, kappa=1.0):
#    generator.load_state_dict(torch.load("results/generator"))
#    discriminator.load_state_dict(torch.load("results/discriminator"))

    generator.to(device).eval()
    discriminator.to(device).eval()
    encoder.to(device)

    criterion = nn.MSELoss()

    optimizer_E = torch.optim.Adam(encoder.parameters(),
                                   lr=opt.lr, betas=(opt.b1, opt.b2))

    os.makedirs("results/images_e", exist_ok=True)

    padding_epoch = len(str(opt.n_epochs))
    padding_i = len(str(len(dataloader)))

    batches_done = 0
    for epoch in range(opt.n_epochs):
        for i, (imgs, _) in enumerate(dataloader):

            # Configure input
            real_imgs = imgs.to(device)

            # ----------------
            #  Train Encoder
            # ----------------

            optimizer_E.zero_grad()

            # Generate a batch of latent variables
            z = encoder(real_imgs)

            # Generate a batch of images
            fake_imgs = generator(z)

            # Real features
            real_features = discriminator.forward_features(real_imgs)
            # Fake features
            fake_features = discriminator.forward_features(fake_imgs)

            # izif architecture
            loss_imgs = criterion(fake_imgs, real_imgs)
            loss_features = criterion(fake_features, real_features)
            e_loss = loss_imgs + kappa * loss_features

            e_loss.backward()
            optimizer_E.step()

            # Output training log every n_critic steps
            if i % opt.n_critic == 0:
                print(f"[Epoch {epoch:{padding_epoch}}/{opt.n_epochs}] "
                      f"[Batch {i:{padding_i}}/{len(dataloader)}] "
                      f"[E loss: {e_loss.item():3f}]")

                if batches_done % opt.sample_interval == 0:
                    fake_z = encoder(fake_imgs)
                    reconfiguration_imgs = generator(fake_z)
                    save_image(reconfiguration_imgs.data[:25],
                               f"results/images_e/{batches_done:06}.png",
                               nrow=5, normalize=True)

                batches_done += opt.n_critic
    torch.save(encoder.state_dict(), "results/encoder")


In [189]:
def test_anomaly_breast(opt, generator, discriminator, encoder, dataloader, device, kappa=1.0, title="all"):
#    generator.load_state_dict(torch.load("results/generator"))
#    discriminator.load_state_dict(torch.load("results/discriminator"))
#    encoder.load_state_dict(torch.load("results/encoder"))

    generator.to(device).eval()
    discriminator.to(device).eval()
    encoder.to(device).eval()

    criterion = nn.MSELoss()

    with open("results/score_"+title+".csv", "w") as f:
        f.write("label, img_distance,anomaly_score,z_distance\n")

    for i, (img, label) in enumerate(dataloader):

        real_img = img.to(device)

        real_z = encoder(real_img)
        fake_img = generator(real_z)
        fake_z = encoder(fake_img)

        real_feature = discriminator.forward_features(real_img)
        fake_feature = discriminator.forward_features(fake_img)

        # Scores for anomaly detection
        img_distance = criterion(fake_img, real_img)
        loss_feature = criterion(fake_feature, real_feature)
        #anomaly_score = img_distance + kappa * loss_feature
        anomaly_score = 50*img_distance + 950*loss_feature
        

        z_distance = criterion(fake_z, real_z)

        with open("results/score_"+title+".csv", "a") as f:
            f.write(f"{label[0]},{img_distance},"
                    f"{anomaly_score},{z_distance}\n")

        #with open("results/score.csv", "a") as f:
        #    f.write(f"{label.item()},{img_distance},"
        #            f"{anomaly_score},{z_distance}\n")


In [139]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=7000,
                        help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=32,
                        help="size of the batches")
parser.add_argument("--lr", type=float, default=0.001,
                        help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5,
                        help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999,
                        help="adam: decay of first order momentum of gradient")
parser.add_argument("--latent_dim", type=int, default=128,
                        help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=256,
                        help="size of each image dimension")
parser.add_argument("--channels", type=int, default=3,
                        help="number of image channels")
parser.add_argument("--n_critic", type=int, default=1000,
                        help="number of training steps for "
                             "discriminator per iter")
parser.add_argument("--sample_interval", type=int, default=5000,
                        help="interval betwen image samples")
parser.add_argument("--seed", type=int, default=31,
                        help="value of a random seed")

opt = parser.parse_args(args=[])


In [140]:
if type(opt.seed) is int:
        torch.manual_seed(opt.seed)
    

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([transforms.Resize([opt.img_size]*2),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.5, 0.5, 0.5],
                                                         [0.5, 0.5, 0.5])])
data_dir = '../anoDataset'
generator = Generator(opt)
discriminator = Discriminator(opt)
encoder = Encoder(opt)

In [26]:
#### Train

anoDataset1 = BreastDataset(data_dir, transform, './Lightshot_normal_img')
print(len(anoDataset1))
train_dataloader = DataLoader(anoDataset1, batch_size=opt.batch_size,shuffle=True)


251


In [14]:
train_wgangp(opt, generator, discriminator, train_dataloader, device)

251
[Epoch    0/7000] [Batch 0/8] [D loss: 9.247296] [G loss: -0.001666]
[Epoch    1/7000] [Batch 0/8] [D loss: -41.272041] [G loss: -1.779982]
[Epoch    2/7000] [Batch 0/8] [D loss: -120.270798] [G loss: -8.003080]
[Epoch    3/7000] [Batch 0/8] [D loss: -117.245674] [G loss: -9.176527]
[Epoch    4/7000] [Batch 0/8] [D loss: -135.334137] [G loss: -26.308910]
[Epoch    5/7000] [Batch 0/8] [D loss: -119.734093] [G loss: -29.873253]
[Epoch    6/7000] [Batch 0/8] [D loss: -122.076225] [G loss: -48.255760]
[Epoch    7/7000] [Batch 0/8] [D loss: -110.271439] [G loss: -67.134109]
[Epoch    8/7000] [Batch 0/8] [D loss: -131.751556] [G loss: -86.228500]
[Epoch    9/7000] [Batch 0/8] [D loss: -124.331223] [G loss: -94.371109]
[Epoch   10/7000] [Batch 0/8] [D loss: -136.664108] [G loss: -137.645660]
[Epoch   11/7000] [Batch 0/8] [D loss: -113.227341] [G loss: -117.259140]
[Epoch   12/7000] [Batch 0/8] [D loss: -96.614319] [G loss: -162.629333]
[Epoch   13/7000] [Batch 0/8] [D loss: -62.363873] [G

[Epoch  114/7000] [Batch 0/8] [D loss: -26.423679] [G loss: -10.974901]
[Epoch  115/7000] [Batch 0/8] [D loss: -19.565565] [G loss: 0.956152]
[Epoch  116/7000] [Batch 0/8] [D loss: -30.675713] [G loss: -36.108295]
[Epoch  117/7000] [Batch 0/8] [D loss: -31.254656] [G loss: -22.612133]
[Epoch  118/7000] [Batch 0/8] [D loss: -27.491423] [G loss: 14.859423]
[Epoch  119/7000] [Batch 0/8] [D loss: -34.032051] [G loss: -10.901875]
[Epoch  120/7000] [Batch 0/8] [D loss: -19.813232] [G loss: -39.420769]
[Epoch  121/7000] [Batch 0/8] [D loss: -26.907379] [G loss: -36.311852]
[Epoch  122/7000] [Batch 0/8] [D loss: -31.582811] [G loss: -53.731850]
[Epoch  123/7000] [Batch 0/8] [D loss: -30.045059] [G loss: -29.829853]
[Epoch  124/7000] [Batch 0/8] [D loss: -25.378191] [G loss: 31.597994]
[Epoch  125/7000] [Batch 0/8] [D loss: -18.728270] [G loss: -30.711948]
[Epoch  126/7000] [Batch 0/8] [D loss: -45.177883] [G loss: -56.124496]
[Epoch  127/7000] [Batch 0/8] [D loss: -20.414619] [G loss: -13.4015

[Epoch  229/7000] [Batch 0/8] [D loss: -8.043526] [G loss: -18.827297]
[Epoch  230/7000] [Batch 0/8] [D loss: -16.943018] [G loss: -87.597412]
[Epoch  231/7000] [Batch 0/8] [D loss: -5.532952] [G loss: -56.968323]
[Epoch  232/7000] [Batch 0/8] [D loss: -22.127369] [G loss: 30.606976]
[Epoch  233/7000] [Batch 0/8] [D loss: 2.780735] [G loss: 19.700430]
[Epoch  234/7000] [Batch 0/8] [D loss: -0.277755] [G loss: -75.064857]
[Epoch  235/7000] [Batch 0/8] [D loss: -42.078026] [G loss: -80.753662]
[Epoch  236/7000] [Batch 0/8] [D loss: -34.364529] [G loss: 18.779779]
[Epoch  237/7000] [Batch 0/8] [D loss: 3.475352] [G loss: 15.724937]
[Epoch  238/7000] [Batch 0/8] [D loss: -2.608636] [G loss: -60.544285]
[Epoch  239/7000] [Batch 0/8] [D loss: -34.908993] [G loss: -47.704659]
[Epoch  240/7000] [Batch 0/8] [D loss: -14.914135] [G loss: 40.748333]
[Epoch  241/7000] [Batch 0/8] [D loss: -3.272876] [G loss: 23.972845]
[Epoch  242/7000] [Batch 0/8] [D loss: -13.712883] [G loss: -9.075367]
[Epoch  

[Epoch  345/7000] [Batch 0/8] [D loss: 1.268219] [G loss: -11.196221]
[Epoch  346/7000] [Batch 0/8] [D loss: -14.909565] [G loss: 1.386972]
[Epoch  347/7000] [Batch 0/8] [D loss: -20.134655] [G loss: 21.881849]
[Epoch  348/7000] [Batch 0/8] [D loss: -7.903730] [G loss: 41.472313]
[Epoch  349/7000] [Batch 0/8] [D loss: -4.751439] [G loss: 48.034821]
[Epoch  350/7000] [Batch 0/8] [D loss: -11.918146] [G loss: -9.072412]
[Epoch  351/7000] [Batch 0/8] [D loss: -20.376091] [G loss: -23.062151]
[Epoch  352/7000] [Batch 0/8] [D loss: -23.328297] [G loss: -18.139450]
[Epoch  353/7000] [Batch 0/8] [D loss: -7.846916] [G loss: 23.707405]
[Epoch  354/7000] [Batch 0/8] [D loss: -26.359423] [G loss: 19.116955]
[Epoch  355/7000] [Batch 0/8] [D loss: -22.725368] [G loss: 22.942720]
[Epoch  356/7000] [Batch 0/8] [D loss: -30.811935] [G loss: 13.089211]
[Epoch  357/7000] [Batch 0/8] [D loss: 6.116616] [G loss: 17.968611]
[Epoch  358/7000] [Batch 0/8] [D loss: -16.997044] [G loss: 25.347485]
[Epoch  359

[Epoch  462/7000] [Batch 0/8] [D loss: -4.257037] [G loss: 4.275809]
[Epoch  463/7000] [Batch 0/8] [D loss: -32.628922] [G loss: -37.560013]
[Epoch  464/7000] [Batch 0/8] [D loss: -0.526607] [G loss: -40.498463]
[Epoch  465/7000] [Batch 0/8] [D loss: -3.767539] [G loss: -10.569049]
[Epoch  466/7000] [Batch 0/8] [D loss: 5.889698] [G loss: 16.302277]
[Epoch  467/7000] [Batch 0/8] [D loss: -21.548365] [G loss: 23.467239]
[Epoch  468/7000] [Batch 0/8] [D loss: -8.528202] [G loss: 5.826358]
[Epoch  469/7000] [Batch 0/8] [D loss: -15.142633] [G loss: -20.901920]
[Epoch  470/7000] [Batch 0/8] [D loss: -13.087417] [G loss: -22.005039]
[Epoch  471/7000] [Batch 0/8] [D loss: 3.875161] [G loss: -22.809937]
[Epoch  472/7000] [Batch 0/8] [D loss: -2.527368] [G loss: -3.342533]
[Epoch  473/7000] [Batch 0/8] [D loss: -19.038183] [G loss: 17.962690]
[Epoch  474/7000] [Batch 0/8] [D loss: -27.891897] [G loss: 24.175543]
[Epoch  475/7000] [Batch 0/8] [D loss: -13.340275] [G loss: 0.561629]
[Epoch  476/

...

[Epoch 6516/7000] [Batch 0/8] [D loss: -55.041515] [G loss: 189.887589]
[Epoch 6517/7000] [Batch 0/8] [D loss: 23.742462] [G loss: 299.969727]
[Epoch 6518/7000] [Batch 0/8] [D loss: 38.645756] [G loss: 247.309326]
[Epoch 6519/7000] [Batch 0/8] [D loss: 0.452305] [G loss: 260.898254]
[Epoch 6520/7000] [Batch 0/8] [D loss: 49.365715] [G loss: 268.356506]
[Epoch 6521/7000] [Batch 0/8] [D loss: 0.690405] [G loss: 293.949585]
[Epoch 6522/7000] [Batch 0/8] [D loss: -50.554298] [G loss: 291.766266]
[Epoch 6523/7000] [Batch 0/8] [D loss: 20.210735] [G loss: 220.044098]
[Epoch 6524/7000] [Batch 0/8] [D loss: 38.232738] [G loss: 266.782166]
[Epoch 6525/7000] [Batch 0/8] [D loss: 32.763966] [G loss: 233.345123]
[Epoch 6526/7000] [Batch 0/8] [D loss: -4.947021] [G loss: 231.829773]
[Epoch 6527/7000] [Batch 0/8] [D loss: 20.423729] [G loss: 248.025970]
[Epoch 6528/7000] [Batch 0/8] [D loss: 29.526068] [G loss: 234.137512]
[Epoch 6529/7000] [Batch 0/8] [D loss: 66.791069] [G loss: 308.291534]
[Epoch

[Epoch 6631/7000] [Batch 0/8] [D loss: 3.525527] [G loss: 227.593307]
[Epoch 6632/7000] [Batch 0/8] [D loss: -32.421570] [G loss: 251.839996]
[Epoch 6633/7000] [Batch 0/8] [D loss: 15.579653] [G loss: 232.349045]
[Epoch 6634/7000] [Batch 0/8] [D loss: -28.070000] [G loss: 264.149414]
[Epoch 6635/7000] [Batch 0/8] [D loss: -34.627182] [G loss: 247.738678]
[Epoch 6636/7000] [Batch 0/8] [D loss: -25.398306] [G loss: 221.656845]
[Epoch 6637/7000] [Batch 0/8] [D loss: 44.345703] [G loss: 210.971222]
[Epoch 6638/7000] [Batch 0/8] [D loss: 17.187597] [G loss: 252.596054]
[Epoch 6639/7000] [Batch 0/8] [D loss: 32.778896] [G loss: 240.930771]
[Epoch 6640/7000] [Batch 0/8] [D loss: -3.557566] [G loss: 220.195953]
[Epoch 6641/7000] [Batch 0/8] [D loss: 20.446955] [G loss: 254.642975]
[Epoch 6642/7000] [Batch 0/8] [D loss: 39.020882] [G loss: 311.412109]
[Epoch 6643/7000] [Batch 0/8] [D loss: 34.670914] [G loss: 293.045624]
[Epoch 6644/7000] [Batch 0/8] [D loss: 32.407642] [G loss: 227.035706]
[Ep

[Epoch 6746/7000] [Batch 0/8] [D loss: 14.066088] [G loss: 228.819092]
[Epoch 6747/7000] [Batch 0/8] [D loss: -48.994995] [G loss: 219.029221]
[Epoch 6748/7000] [Batch 0/8] [D loss: -9.842827] [G loss: 271.973145]
[Epoch 6749/7000] [Batch 0/8] [D loss: -16.384172] [G loss: 243.597076]
[Epoch 6750/7000] [Batch 0/8] [D loss: -59.460190] [G loss: 214.243454]
[Epoch 6751/7000] [Batch 0/8] [D loss: 50.843597] [G loss: 227.821152]
[Epoch 6752/7000] [Batch 0/8] [D loss: 21.625881] [G loss: 215.832672]
[Epoch 6753/7000] [Batch 0/8] [D loss: -28.375856] [G loss: 224.381134]
[Epoch 6754/7000] [Batch 0/8] [D loss: -36.919056] [G loss: 214.879776]
[Epoch 6755/7000] [Batch 0/8] [D loss: 22.680277] [G loss: 218.781494]
[Epoch 6756/7000] [Batch 0/8] [D loss: -13.307256] [G loss: 262.577454]
[Epoch 6757/7000] [Batch 0/8] [D loss: -26.492645] [G loss: 273.310120]
[Epoch 6758/7000] [Batch 0/8] [D loss: -40.436867] [G loss: 231.190933]
[Epoch 6759/7000] [Batch 0/8] [D loss: -37.949818] [G loss: 229.94543

[Epoch 6861/7000] [Batch 0/8] [D loss: -34.569946] [G loss: 243.144348]
[Epoch 6862/7000] [Batch 0/8] [D loss: -15.713931] [G loss: 261.943542]
[Epoch 6863/7000] [Batch 0/8] [D loss: 49.931458] [G loss: 253.863556]
[Epoch 6864/7000] [Batch 0/8] [D loss: 31.111107] [G loss: 300.146790]
[Epoch 6865/7000] [Batch 0/8] [D loss: 18.640116] [G loss: 284.205444]
[Epoch 6866/7000] [Batch 0/8] [D loss: 10.159259] [G loss: 261.961792]
[Epoch 6867/7000] [Batch 0/8] [D loss: -34.658974] [G loss: 287.463867]
[Epoch 6868/7000] [Batch 0/8] [D loss: 48.824257] [G loss: 293.533020]
[Epoch 6869/7000] [Batch 0/8] [D loss: 56.443668] [G loss: 287.258026]
[Epoch 6870/7000] [Batch 0/8] [D loss: 23.842295] [G loss: 271.592682]
[Epoch 6871/7000] [Batch 0/8] [D loss: -39.298809] [G loss: 245.823105]
[Epoch 6872/7000] [Batch 0/8] [D loss: -31.119131] [G loss: 248.104370]
[Epoch 6873/7000] [Batch 0/8] [D loss: -35.080685] [G loss: 227.790176]
[Epoch 6874/7000] [Batch 0/8] [D loss: -43.094963] [G loss: 249.602234]

[Epoch 6976/7000] [Batch 0/8] [D loss: -47.070595] [G loss: 192.963440]
[Epoch 6977/7000] [Batch 0/8] [D loss: -35.838924] [G loss: 242.884277]
[Epoch 6978/7000] [Batch 0/8] [D loss: 46.180943] [G loss: 234.494110]
[Epoch 6979/7000] [Batch 0/8] [D loss: -23.721992] [G loss: 238.330612]
[Epoch 6980/7000] [Batch 0/8] [D loss: 24.572493] [G loss: 224.702347]
[Epoch 6981/7000] [Batch 0/8] [D loss: 1.273681] [G loss: 258.674194]
[Epoch 6982/7000] [Batch 0/8] [D loss: 7.740292] [G loss: 214.809601]
[Epoch 6983/7000] [Batch 0/8] [D loss: 12.354470] [G loss: 215.090500]
[Epoch 6984/7000] [Batch 0/8] [D loss: -13.032214] [G loss: 253.491104]
[Epoch 6985/7000] [Batch 0/8] [D loss: 2.713622] [G loss: 262.330261]
[Epoch 6986/7000] [Batch 0/8] [D loss: 9.140837] [G loss: 258.418945]
[Epoch 6987/7000] [Batch 0/8] [D loss: 17.943909] [G loss: 227.422501]
[Epoch 6988/7000] [Batch 0/8] [D loss: 41.283039] [G loss: 254.547546]
[Epoch 6989/7000] [Batch 0/8] [D loss: -0.348657] [G loss: 223.645844]
[Epoch

In [24]:
generator.load_state_dict(torch.load("results/generator"))
discriminator.load_state_dict(torch.load("results/discriminator"))
# Encoder 만 1000번 배치사이즈 16, 왜냐하면 GPU 메모리 모자라다고 해서 다시 시작하느라..

<All keys matched successfully>

In [27]:
train_encoder_izif(opt, generator, discriminator, encoder,train_dataloader, device)

[Epoch    0/1000] [Batch  0/16] [E loss: 0.093992]
[Epoch    1/1000] [Batch  0/16] [E loss: 0.064454]
[Epoch    2/1000] [Batch  0/16] [E loss: 0.055175]
[Epoch    3/1000] [Batch  0/16] [E loss: 0.050737]
[Epoch    4/1000] [Batch  0/16] [E loss: 0.046241]
[Epoch    5/1000] [Batch  0/16] [E loss: 0.044924]
[Epoch    6/1000] [Batch  0/16] [E loss: 0.039119]
[Epoch    7/1000] [Batch  0/16] [E loss: 0.041538]
[Epoch    8/1000] [Batch  0/16] [E loss: 0.036590]
[Epoch    9/1000] [Batch  0/16] [E loss: 0.040180]
[Epoch   10/1000] [Batch  0/16] [E loss: 0.046634]
[Epoch   11/1000] [Batch  0/16] [E loss: 0.039377]
[Epoch   12/1000] [Batch  0/16] [E loss: 0.038316]
[Epoch   13/1000] [Batch  0/16] [E loss: 0.040570]
[Epoch   14/1000] [Batch  0/16] [E loss: 0.041559]
[Epoch   15/1000] [Batch  0/16] [E loss: 0.038055]
[Epoch   16/1000] [Batch  0/16] [E loss: 0.037718]
[Epoch   17/1000] [Batch  0/16] [E loss: 0.041924]
[Epoch   18/1000] [Batch  0/16] [E loss: 0.045844]
[Epoch   19/1000] [Batch  0/16]

[Epoch  161/1000] [Batch  0/16] [E loss: 0.031975]
[Epoch  162/1000] [Batch  0/16] [E loss: 0.029405]
[Epoch  163/1000] [Batch  0/16] [E loss: 0.028076]
[Epoch  164/1000] [Batch  0/16] [E loss: 0.027747]
[Epoch  165/1000] [Batch  0/16] [E loss: 0.029007]
[Epoch  166/1000] [Batch  0/16] [E loss: 0.029547]
[Epoch  167/1000] [Batch  0/16] [E loss: 0.031105]
[Epoch  168/1000] [Batch  0/16] [E loss: 0.027590]
[Epoch  169/1000] [Batch  0/16] [E loss: 0.028983]
[Epoch  170/1000] [Batch  0/16] [E loss: 0.033493]
[Epoch  171/1000] [Batch  0/16] [E loss: 0.030773]
[Epoch  172/1000] [Batch  0/16] [E loss: 0.033449]
[Epoch  173/1000] [Batch  0/16] [E loss: 0.028283]
[Epoch  174/1000] [Batch  0/16] [E loss: 0.026523]
[Epoch  175/1000] [Batch  0/16] [E loss: 0.030723]
[Epoch  176/1000] [Batch  0/16] [E loss: 0.028935]
[Epoch  177/1000] [Batch  0/16] [E loss: 0.029036]
[Epoch  178/1000] [Batch  0/16] [E loss: 0.029847]
[Epoch  179/1000] [Batch  0/16] [E loss: 0.028719]
[Epoch  180/1000] [Batch  0/16]

[Epoch  322/1000] [Batch  0/16] [E loss: 0.027714]
[Epoch  323/1000] [Batch  0/16] [E loss: 0.026805]
[Epoch  324/1000] [Batch  0/16] [E loss: 0.030074]
[Epoch  325/1000] [Batch  0/16] [E loss: 0.029844]
[Epoch  326/1000] [Batch  0/16] [E loss: 0.025463]
[Epoch  327/1000] [Batch  0/16] [E loss: 0.029827]
[Epoch  328/1000] [Batch  0/16] [E loss: 0.032140]
[Epoch  329/1000] [Batch  0/16] [E loss: 0.028237]
[Epoch  330/1000] [Batch  0/16] [E loss: 0.030809]
[Epoch  331/1000] [Batch  0/16] [E loss: 0.029190]
[Epoch  332/1000] [Batch  0/16] [E loss: 0.028097]
[Epoch  333/1000] [Batch  0/16] [E loss: 0.026086]
[Epoch  334/1000] [Batch  0/16] [E loss: 0.028281]
[Epoch  335/1000] [Batch  0/16] [E loss: 0.024786]
[Epoch  336/1000] [Batch  0/16] [E loss: 0.028921]
[Epoch  337/1000] [Batch  0/16] [E loss: 0.023981]
[Epoch  338/1000] [Batch  0/16] [E loss: 0.027755]
[Epoch  339/1000] [Batch  0/16] [E loss: 0.029087]
[Epoch  340/1000] [Batch  0/16] [E loss: 0.031196]
[Epoch  341/1000] [Batch  0/16]

[Epoch  483/1000] [Batch  0/16] [E loss: 0.027991]
[Epoch  484/1000] [Batch  0/16] [E loss: 0.026153]
[Epoch  485/1000] [Batch  0/16] [E loss: 0.029698]
[Epoch  486/1000] [Batch  0/16] [E loss: 0.030939]
[Epoch  487/1000] [Batch  0/16] [E loss: 0.028454]
[Epoch  488/1000] [Batch  0/16] [E loss: 0.029542]
[Epoch  489/1000] [Batch  0/16] [E loss: 0.029136]
[Epoch  490/1000] [Batch  0/16] [E loss: 0.029439]
[Epoch  491/1000] [Batch  0/16] [E loss: 0.026181]
[Epoch  492/1000] [Batch  0/16] [E loss: 0.026454]
[Epoch  493/1000] [Batch  0/16] [E loss: 0.031069]
[Epoch  494/1000] [Batch  0/16] [E loss: 0.028538]
[Epoch  495/1000] [Batch  0/16] [E loss: 0.028482]
[Epoch  496/1000] [Batch  0/16] [E loss: 0.034341]
[Epoch  497/1000] [Batch  0/16] [E loss: 0.027268]
[Epoch  498/1000] [Batch  0/16] [E loss: 0.024898]
[Epoch  499/1000] [Batch  0/16] [E loss: 0.027114]
[Epoch  500/1000] [Batch  0/16] [E loss: 0.026155]
[Epoch  501/1000] [Batch  0/16] [E loss: 0.026007]
[Epoch  502/1000] [Batch  0/16]

[Epoch  644/1000] [Batch  0/16] [E loss: 0.029935]
[Epoch  645/1000] [Batch  0/16] [E loss: 0.024673]
[Epoch  646/1000] [Batch  0/16] [E loss: 0.027750]
[Epoch  647/1000] [Batch  0/16] [E loss: 0.030331]
[Epoch  648/1000] [Batch  0/16] [E loss: 0.029448]
[Epoch  649/1000] [Batch  0/16] [E loss: 0.031463]
[Epoch  650/1000] [Batch  0/16] [E loss: 0.027956]
[Epoch  651/1000] [Batch  0/16] [E loss: 0.029589]
[Epoch  652/1000] [Batch  0/16] [E loss: 0.029155]
[Epoch  653/1000] [Batch  0/16] [E loss: 0.027110]
[Epoch  654/1000] [Batch  0/16] [E loss: 0.028464]
[Epoch  655/1000] [Batch  0/16] [E loss: 0.025313]
[Epoch  656/1000] [Batch  0/16] [E loss: 0.028382]
[Epoch  657/1000] [Batch  0/16] [E loss: 0.028114]
[Epoch  658/1000] [Batch  0/16] [E loss: 0.026477]
[Epoch  659/1000] [Batch  0/16] [E loss: 0.024531]
[Epoch  660/1000] [Batch  0/16] [E loss: 0.029888]
[Epoch  661/1000] [Batch  0/16] [E loss: 0.028145]
[Epoch  662/1000] [Batch  0/16] [E loss: 0.027141]
[Epoch  663/1000] [Batch  0/16]

[Epoch  805/1000] [Batch  0/16] [E loss: 0.025534]
[Epoch  806/1000] [Batch  0/16] [E loss: 0.026904]
[Epoch  807/1000] [Batch  0/16] [E loss: 0.027067]
[Epoch  808/1000] [Batch  0/16] [E loss: 0.026122]
[Epoch  809/1000] [Batch  0/16] [E loss: 0.027892]
[Epoch  810/1000] [Batch  0/16] [E loss: 0.029546]
[Epoch  811/1000] [Batch  0/16] [E loss: 0.030211]
[Epoch  812/1000] [Batch  0/16] [E loss: 0.024075]
[Epoch  813/1000] [Batch  0/16] [E loss: 0.027169]
[Epoch  814/1000] [Batch  0/16] [E loss: 0.029512]
[Epoch  815/1000] [Batch  0/16] [E loss: 0.025607]
[Epoch  816/1000] [Batch  0/16] [E loss: 0.030101]
[Epoch  817/1000] [Batch  0/16] [E loss: 0.031720]
[Epoch  818/1000] [Batch  0/16] [E loss: 0.028641]
[Epoch  819/1000] [Batch  0/16] [E loss: 0.026975]
[Epoch  820/1000] [Batch  0/16] [E loss: 0.026455]
[Epoch  821/1000] [Batch  0/16] [E loss: 0.023824]
[Epoch  822/1000] [Batch  0/16] [E loss: 0.029098]
[Epoch  823/1000] [Batch  0/16] [E loss: 0.027594]
[Epoch  824/1000] [Batch  0/16]

[Epoch  966/1000] [Batch  0/16] [E loss: 0.026698]
[Epoch  967/1000] [Batch  0/16] [E loss: 0.026256]
[Epoch  968/1000] [Batch  0/16] [E loss: 0.027554]
[Epoch  969/1000] [Batch  0/16] [E loss: 0.030103]
[Epoch  970/1000] [Batch  0/16] [E loss: 0.028302]
[Epoch  971/1000] [Batch  0/16] [E loss: 0.026324]
[Epoch  972/1000] [Batch  0/16] [E loss: 0.024793]
[Epoch  973/1000] [Batch  0/16] [E loss: 0.028103]
[Epoch  974/1000] [Batch  0/16] [E loss: 0.026391]
[Epoch  975/1000] [Batch  0/16] [E loss: 0.028795]
[Epoch  976/1000] [Batch  0/16] [E loss: 0.027522]
[Epoch  977/1000] [Batch  0/16] [E loss: 0.026436]
[Epoch  978/1000] [Batch  0/16] [E loss: 0.025869]
[Epoch  979/1000] [Batch  0/16] [E loss: 0.031030]
[Epoch  980/1000] [Batch  0/16] [E loss: 0.025499]
[Epoch  981/1000] [Batch  0/16] [E loss: 0.026427]
[Epoch  982/1000] [Batch  0/16] [E loss: 0.024985]
[Epoch  983/1000] [Batch  0/16] [E loss: 0.023760]
[Epoch  984/1000] [Batch  0/16] [E loss: 0.027549]
[Epoch  985/1000] [Batch  0/16]

In [172]:
#### Test
generator.load_state_dict(torch.load("results/generator"))
discriminator.load_state_dict(torch.load("results/discriminator"))
encoder.load_state_dict(torch.load("results/encoder"))

<All keys matched successfully>

In [190]:
anoDataset2 = BreastDataset(data_dir, transform, './Lightshot_preRT_img')
print(len(anoDataset2))
test_dataloader = DataLoader(anoDataset2, batch_size=1, shuffle=False)
test_anomaly_breast(opt, generator, discriminator, encoder, test_dataloader, device, kappa=10, title="preRT")

anoDataset2 = BreastDataset(data_dir, transform, './Lightshot_postRT_img')
print(len(anoDataset2))
test_dataloader = DataLoader(anoDataset2, batch_size=1, shuffle=False)
test_anomaly_breast(opt, generator, discriminator, encoder, test_dataloader, device, kappa=10, title="postRT")


176
112


In [191]:
import sklearn
from sklearn.preprocessing import StandardScaler

anoDataset1 = BreastDataset(data_dir, transform, './Lightshot_normal_img')
train_dataloader = DataLoader(anoDataset1, batch_size=1,shuffle=False)
test_anomaly_breast(opt, generator, discriminator, encoder, train_dataloader, device, kappa=10, title="train")

train_score = pd.read_csv('results/score_train.csv')

transformer = StandardScaler()
transformer.fit(train_score[['anomaly_score']])
train_score['normal_z'] = transformer.transform(train_score[['anomaly_score']])

train_score.to_csv('results/zscore_normal.csv')


In [192]:
pre_score = pd.read_csv('results/score_preRT.csv')
pre_score['anomaly_z']=transformer.transform(pre_score[['anomaly_score']])

post_score = pd.read_csv('results/score_postRT.csv')
post_score['anomaly_z']=transformer.transform(post_score[['anomaly_score']])

pre_score.to_csv('results/zscore_preRT.csv', index=False)
post_score.to_csv('results/zscore_postRT.csv', index=False)


# 